# 0. Libraries and importing the data

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import itertools
import math
from scipy import stats

We import the training and the test data:

In [23]:
# We import the training dataset
df_og = pd.read_csv('../train.csv')

# We create a copy of the original dataset, so that we do not have to load the dataset
# again if we want to go back to the original one
df = df_og.copy()

# We import the training dataset
dft_og = pd.read_csv('../test.csv')

# We create a copy of the original dataset, so that we do not have to load the dataset
# again if we want to go back to the original one
dft = dft_og.copy()

In [5]:
# First, we reorder the dataframe and assign some prefixes to the positions
# so that it is easier to catch specific patterns

# Mapping positions to their groups
group_mapping = {
    'Goalkeeper': ['GK'],
    'Defenders': ['CB', 'LCB', 'RCB', 'LB', 'RB', 'LWB', 'RWB'],
    'Midfielders': ['CDM', 'LDM', 'RDM', 'CM', 'LCM', 'RCM', 'CAM', 'LM', 'RM'],
    'Attackers': ['ST', 'RS', 'LS', 'LF', 'RF', 'RW', 'LW']
}

# Prefix mappings
prefix_mapping = {
    'Goalkeeper': 'G_',
    'Defenders': 'D_',
    'Midfielders': 'M_',
    'Attackers': 'A_'
}

# Flatten mapping to find the group for each position
position_to_group = {pos: group for group, positions in group_mapping.items() for pos in positions}

# Adding a prefix to position names
df['position_grouped'] = df['position'].map(lambda x: prefix_mapping[position_to_group[x]] + x)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6895 entries, 0 to 6894
Data columns (total 71 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           6895 non-null   int64  
 1   short_name                   6895 non-null   object 
 2   overall                      6895 non-null   int64  
 3   potential                    6895 non-null   int64  
 4   value_eur                    6893 non-null   float64
 5   wage_eur                     6895 non-null   int64  
 6   birthday_date                6895 non-null   object 
 7   height_cm                    6895 non-null   int64  
 8   weight_kg                    6895 non-null   int64  
 9   club_name                    6895 non-null   object 
 10  league_name                  6895 non-null   object 
 11  league_level                 6895 non-null   int64  
 12  club_jersey_number           6895 non-null   int64  
 13  club_loaned_from  

In [ ]:
# List of numerical features (50 in total)
numerical_features = [
    'overall', 'potential', 'wage_eur', 'height_cm', 'weight_kg',
    'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys',
    'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
    'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance',
    'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
    'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
    'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle',
    'goalkeeping_speed', 'value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 
    'physic', 'months_since_joined', 'age'
]

# List of date features (1 in total)
date_features = ['club_contract_valid_until']

# List of categorical ordinal features (5 in total)
categorical_ordinal_features = [
    'league_level', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate'
]

# List of categorical nominal features (13 in total)
categorical_nominal_features = [
    'club_jersey_number', 'short_name', 'club_name', 
    'club_loaned_from', 'nationality_name', 'preferred_foot', 'body_type', 'real_face', 
    'nation_jersey_number', 'league_country'
]

# Complete list of categorical features
categorical_features = categorical_nominal_features + categorical_ordinal_features

# From the list of categorical features, we only keep those for which the visualization makes sense
# (i.e., they have a low enough number of distinct values)
categorical_features_eda = categorical_ordinal_features + ['preferred_foot', 'body_type']

# 2. Preprocessing

## 2.1. Feature creation
Create features that may be useful for improving the predictions (before splitting the data *in case that feature creation is NOT done taking into consideration the specific values of the dataset*). Features created/replaced, for now (for more suggestions, refer to the EDA notebook):
- Replace *league_name* with the country of the league: that way, together with *league_level*, information will be more efficient.
- For dealing with dates more easily, we can convert the features *club_joined* and *birthday_date* to the number of months since a player joined a club and the age in years, respectively. Then, we can drop the other features.

**Also, we have to consider that there might be different unique values in the test set than in the training set, so the feature creation in these cases must be done with the training and the test set together!**

In [24]:
# Combine train and test datasets. We will use this dataset to create features
# that depend on the unique values of categories (such as league_name)
df_combined = pd.concat([df, dft], ignore_index=True)

### Replacing *league_name* with *league_country*

In [25]:
# List of unique league_name
print(df_combined.league_name.unique())
print(df_combined.league_name.nunique())

['Mexican Liga MX' 'Scottish Premiership' 'Danish Superliga'
 'Korean K League 1' 'Romanian Liga I' 'Peruvian Primera División'
 'German 1. Bundesliga' 'Chinese Super League'
 'Belgian Jupiler Pro League' 'English Premier League'
 'German 2. Bundesliga' 'French Ligue 2' 'Argentina Primera División'
 'Paraguayan Primera División' 'Liga de Fútbol Profesional Boliviano'
 'Swedish Allsvenskan' 'Spanish Segunda División'
 'English League Championship' 'Turkish Süper Lig' 'Norwegian Eliteserien'
 'Polish T-Mobile Ekstraklasa' 'Japanese J. League Division 1'
 'German 3. Bundesliga' 'Campeonato Brasileiro Série A'
 'English National League' 'USA Major League Soccer'
 'Colombian Liga Postobón' 'Austrian Football Bundesliga'
 'English League One' 'Saudi Abdul L. Jameel League' 'French Ligue 1'
 'Hungarian Nemzeti Bajnokság I' 'Italian Serie A'
 'Spain Primera Division' 'English League Two'
 'Portuguese Liga ZON SAGRES' 'Australian Hyundai A-League'
 'Ecuadorian Serie A' 'Holland Eredivisie' 'Chi

In [26]:
# We create a map for the league with the corresponding country
league_to_country = {
    'Mexican Liga MX': 'Mexico',
    'Scottish Premiership': 'Scotland',
    'Danish Superliga': 'Denmark',
    'Korean K League 1': 'South Korea',
    'Romanian Liga I': 'Romania',
    'Peruvian Primera División': 'Peru',
    'German 1. Bundesliga': 'Germany',
    'Chinese Super League': 'China',
    'Belgian Jupiler Pro League': 'Belgium',
    'English Premier League': 'England',
    'German 2. Bundesliga': 'Germany',
    'French Ligue 2': 'France',
    'Argentina Primera División': 'Argentina',
    'Paraguayan Primera División': 'Paraguay',
    'Liga de Fútbol Profesional Boliviano': 'Bolivia',
    'Swedish Allsvenskan': 'Sweden',
    'Spanish Segunda División': 'Spain',
    'English League Championship': 'England',
    'Turkish Süper Lig': 'Turkey',
    'Norwegian Eliteserien': 'Norway',
    'Polish T-Mobile Ekstraklasa': 'Poland',
    'Japanese J. League Division 1': 'Japan',
    'German 3. Bundesliga': 'Germany',
    'Campeonato Brasileiro Série A': 'Brazil',
    'English National League': 'England',
    'USA Major League Soccer': 'USA',
    'Colombian Liga Postobón': 'Colombia',
    'Austrian Football Bundesliga': 'Austria',
    'English League One': 'England',
    'Saudi Abdul L. Jameel League': 'Saudi Arabia',
    'French Ligue 1': 'France',
    'Hungarian Nemzeti Bajnokság I': 'Hungary',
    'Italian Serie A': 'Italy',
    'Spain Primera Division': 'Spain',
    'English League Two': 'England',
    'Portuguese Liga ZON SAGRES': 'Portugal',
    'Australian Hyundai A-League': 'Australia',
    'Ecuadorian Serie A': 'Ecuador',
    'Holland Eredivisie': 'Netherlands',
    'Chilian Campeonato Nacional': 'Chile',
    'Rep. Ireland Airtricity League': 'Ireland',
    'Czech Republic Gambrinus Liga': 'Czech Republic',
    'South African Premier Division': 'South Africa',
    'Greek Super League': 'Greece',
    'Russian Premier League': 'Russia',
    'Indian Super League': 'India',
    'Italian Serie B': 'Italy',
    'Swiss Super League': 'Switzerland',
    'Venezuelan Primera División': 'Venezuela',
    'Uruguayan Primera División': 'Uruguay',
    'Ukrainian Premier League': 'Ukraine',
    'UAE Arabian Gulf League': 'UAE',
    'Croatian Prva HNL': 'Croatia',
    'Finnish Veikkausliiga': 'Finland',
    'Cypriot First Division': 'Cyprus'
}

# We create a copy of the training and test datasets
df_league = df.copy()
dft_league = dft.copy()

# We map the league_names with the country and remove the original column, in both
# the training and test datasets
df_league['league_country'] = df_league['league_name'].map(league_to_country)
dft_league['league_country'] = dft_league['league_name'].map(league_to_country)

# Drop the original league_name column
df_league.drop(columns=['league_name'], inplace=True)
dft_league.drop(columns=['league_name'], inplace=True)

In [ ]:
df_league

We have to update the list of feature names with this new feature (see the beginning of the notebook).

### Replacing date variables *club_joined* and *birthday_date*
To the number of months since a player joined a club and the age in years, respectively. Then, we can drop the other features. As a reference date for doing these computations, we can check what is the last date when a player joined a club - if it is in the end of 2022, for instance, then we can take the end of 2022 as the reference date.

In [29]:
df_dates = df_league.copy()
dft_dates = dft_league.copy()

# Convert columns to datetime
## Training
df_dates['club_joined'] = pd.to_datetime(df_dates['club_joined'], format='%Y-%m-%d')
df_dates['birthday_date'] = pd.to_datetime(df_dates['birthday_date'], format='%Y-%m-%d')
## Test
dft_dates['club_joined'] = pd.to_datetime(dft_dates['club_joined'], format='%Y-%m-%d')
dft_dates['birthday_date'] = pd.to_datetime(dft_dates['birthday_date'], format='%Y-%m-%d')
## Combined (only the club_joined is needed for determining the reference date)
df_combined['club_joined'] = pd.to_datetime(df_combined['club_joined'], format='%Y-%m-%d')

# Determine the reference date (latest club_joined date)
reference_date = df_combined['club_joined'].max()

# Compute months since club_joined
## Training
df_dates['months_since_joined'] = (reference_date.year - df_dates['club_joined'].dt.year) * 12 + (reference_date.month - df_dates['club_joined'].dt.month)
## Test
dft_dates['months_since_joined'] = (reference_date.year - dft_dates['club_joined'].dt.year) * 12 + (reference_date.month - dft_dates['club_joined'].dt.month)

# Compute age in years, adjusting if the player's birthday has not occurred yet
## Training
df_dates['age'] = reference_date.year - df_dates['birthday_date'].dt.year - (
    (reference_date.month * 100 + reference_date.day) < (df_dates['birthday_date'].dt.month * 100 + df_dates['birthday_date'].dt.day)
)
## Test
dft_dates['age'] = reference_date.year - dft_dates['birthday_date'].dt.year - (
    (reference_date.month * 100 + reference_date.day) < (dft_dates['birthday_date'].dt.month * 100 + dft_dates['birthday_date'].dt.day)
)

# Drop the original date columns
## Training
df_dates.drop(columns=['club_joined', 'birthday_date'], inplace=True)
## Test
dft_dates.drop(columns=['club_joined', 'birthday_date'], inplace=True)

In [30]:
df_dates

,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_level,...,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,position,league_country,months_since_joined,age
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,1,...,14,11,12,12,12,NaN,LB,Mexico,2.0,31
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,1,...,10,13,7,6,11,NaN,LDM,Scotland,2.0,24
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,1,...,10,10,8,10,11,NaN,LWB,Denmark,2.0,19
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,1,...,16,15,13,8,9,NaN,LS,South Korea,43.0,29
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,1,...,12,5,11,12,15,NaN,LB,Romania,26.0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,4,...,12,9,10,8,13,NaN,RCB,England,2.0,23
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,1,...,16,16,11,6,8,NaN,LCM,Saudi Arabia,2.0,33
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,1,...,8,9,5,8,10,NaN,LWB,Portugal,13.0,22
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,1,...,8,10,10,10,8,NaN,LCB,Japan,80.0,31


We can now treat these new features as numerical variables, so we need to update the lists of feature names (see the beginning of the notebook).

## 2.2. Feature removal
For now, the following features will be removed from the dataset.
- All goalkeeping variables except for *goalkeeping_speed*, as it is the only one with values for goalkeepers and should predict perfectly the goalkeeper class.
- Also, for now we will delete the variables *player_tags* (which has an excessive number of missing values and for which the imputation will be very inaccurate, see EDA) and *player_traits*, which might be a good predictor if correctly treated (e.g., finding the unique traits and OHE those) but it may take significant time.

### Goalkeeping variables

In [ ]:
df_gk = df_dates.copy()
dft_gk = dft_dates.copy()

# Drop the goalkeeper columns
## Training
df_gk.drop(columns=['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes'], inplace=True)
## Test
dft_gk.drop(columns=['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes'], inplace=True)

In [ ]:
df_gk

We have to update the list of feature names with this new feature removal (see the beginning of the notebook).

### *player_tags* and *player_traits*

In [32]:
df_notext = df_gk.copy()
dft_notext = dft_gk.copy()

# Drop the tags and traits columns
## Training
df_notext.drop(columns=['player_tags', 'player_traits'], inplace=True)
## Test
dft_notext.drop(columns=['player_tags', 'player_traits'], inplace=True)

In [33]:
df_notext

,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_speed,position,league_country,months_since_joined,age
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,1,...,58,66,65,66,65,NaN,LB,Mexico,2.0,31
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,1,...,46,59,65,61,58,NaN,LDM,Scotland,2.0,24
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,1,...,31,65,60,58,59,NaN,LWB,Denmark,2.0,19
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,1,...,75,75,16,22,19,NaN,LS,South Korea,43.0,29
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,1,...,52,58,64,61,58,NaN,LB,Romania,26.0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,4,...,49,45,52,57,58,NaN,RCB,England,2.0,23
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,1,...,71,81,80,79,75,NaN,LCM,Saudi Arabia,2.0,33
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,1,...,56,62,62,67,68,NaN,LWB,Portugal,13.0,22
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,1,...,38,60,72,72,70,NaN,LCB,Japan,80.0,31


We have to update the list of feature names with this new feature removal (see the beginning of the notebook).

## 2.3. Dealing with categorical variables
In this section, we apply:
- OHE to categorical features relevant for prediction - those which don't have an enormous number of distinct values (so only *preferred_foot*, *body_type*, *real_face* and *league_country*; maybe also *nationality_name*?).
- Label encoding to the features related to the jersey number (*club_jersey_number* and *nation_jersey_number*).
- Ordinal encoding for all ordinal categorical features.

The same transformation can be applied to the test set without having data leakage.

In [5]:
def ohe_join(dataframe: pd.DataFrame, variables_to_encode: list, na_column: bool):
    for column in variables_to_encode:
        # We encode OHE each variable, creating an additional column which stores NaN
        ohe = pd.get_dummies(data = dataframe[column], drop_first = False, dummy_na = na_column)
        
        # Rename columns to avoid potential conflicts
        ohe.columns = [f"{column}_{str(col)}" for col in ohe.columns]
        
        # Join the encoded variable to the input dataframe
        dataframe = dataframe.join(ohe)
        
        # Drop the original variable from the dataframe
        dataframe = dataframe.drop(labels=column, axis=1)
    
    return dataframe

In [6]:
df3_ohe_test = ohe_join(dataframe = df3, variables_to_encode = ['orientation', 'neighborhood', 'floor'], na_column = True)
df3_ohe_test

,id,num_rooms,num_baths,square_meters,year_built,is_furnished,has_pool,num_crimes,has_ac,accepts_pets,...,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_nan
0,9255,1.0,1.0,NaN,1956.0,False,False,2.0,True,True,...,False,False,True,False,False,False,False,False,False,False
1,1562,4.0,1.0,133.0,1960.0,False,False,4.0,False,False,...,False,False,False,False,False,False,True,False,False,False
2,1671,2.0,3.0,137.0,2000.0,False,True,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6088,1.0,2.0,41.0,2002.0,False,True,5.0,False,False,...,False,False,False,False,False,True,False,False,False,False
4,6670,2.0,1.0,70.0,1979.0,True,False,0.0,False,True,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5735,2.0,NaN,95.0,2021.0,False,False,0.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7996,5192,4.0,3.0,166.0,1995.0,False,False,9.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7997,5391,4.0,1.0,89.0,2015.0,True,False,0.0,True,False,...,False,False,False,False,False,False,False,False,True,False
7998,861,4.0,2.0,167.0,1977.0,False,True,0.0,True,False,...,False,False,False,False,False,False,False,True,False,False


Be mindful that, ***for filtering null values of each of the categorical variables to which OHE has been applied, we must consider their NaN column, where "True" denotes that the corresponding observation had a null value in the categorical variable***. See the example below, where we subtract 8000 (the total number of observations) to the number of `True`s in a column which keeps track of NaN.

In [7]:
8000 - df3_ohe_test['floor_nan'].sum()

7851

In [8]:
print(df3_ohe_test.columns)
len(df3_ohe_test.columns)

Index(['id', 'num_rooms', 'num_baths', 'square_meters', 'year_built',
       'is_furnished', 'has_pool', 'num_crimes', 'has_ac', 'accepts_pets',
       'num_supermarkets', 'price', 'orientation_east', 'orientation_north',
       'orientation_south', 'orientation_west', 'orientation_nan',
       'neighborhood_Ciutat Vella', 'neighborhood_Eixample',
       'neighborhood_Gràcia', 'neighborhood_Horta', 'neighborhood_Les Cors',
       'neighborhood_Nou Barris', 'neighborhood_Sant Andreu',
       'neighborhood_Sant Martí', 'neighborhood_Sants',
       'neighborhood_Sarrià-Sant Gervasi', 'neighborhood_nan', 'floor_1',
       'floor_10', 'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6',
       'floor_7', 'floor_8', 'floor_9', 'floor_nan'],
      dtype='object')


39

---------

In [9]:
df3_ohe_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8000 non-null   int64  
 1   num_rooms                         8000 non-null   float64
 2   num_baths                         7840 non-null   float64
 3   square_meters                     7845 non-null   float64
 4   year_built                        7830 non-null   float64
 5   is_furnished                      7835 non-null   object 
 6   has_pool                          7844 non-null   object 
 7   num_crimes                        7840 non-null   float64
 8   has_ac                            7831 non-null   object 
 9   accepts_pets                      7845 non-null   object 
 10  num_supermarkets                  1411 non-null   float64
 11  price                             8000 non-null   int64  
 12  orient

## 2.4. Splitting the data into a training and a validation set
Consider whether to apply n-fold cross validation here or when training the model. ***See training notebook!!***

## 2.5. Cleaning the data

### 2.4.1. Handling missing values

#### i. Testing whether data is missing completely at random (MCAR), missing at random (MAR) or missing not at random (MNAR)

References:
- https://stefvanbuuren.name/fimd/sec-MCAR.html
- https://campus.datacamp.com/courses/scalable-data-processing-in-r/case-study-a-preliminary-analysis-of-the-housing-data?ex=4

Types of missing data:
- **MCAR**: When data are missing completely at random there is no way to predict where in the data set we'll see a missing value. In an analysis this can often be handled by simply dropping rows of a data set with missing values. 
- **MAR**: When missingness is associated with other variables we call it missing at random. This name is a misnomer. We really mean that *conditioned on some of the variables in the data set, the data are missing completely at random*. To deal with MAR data we generally predict values for the missing data several times (i.e., multiple imputations) to create multiple data sets that capture the statistical structure of the relationships between the variables and then perform an analysis on the data sets. This procedure is called multiple imputation. 
- **MNAR**: The last category, missing not at random is for the case where data is neither MAR nor MCAR. It is usually caused by deterministic relationships between missingness and other measurements. 

There is no direct way to check if the data are MCAR, so, we are going to check if the data are MAR, and if they are not, we will assume that the data are missing completely at random. To check if your data are MAR:
1) Take each column with missingness and recode it as one if it is missing and zero otherwise. 
2) Then regress each of the the other variables onto it using a logistic regression.
3) A significant p-value indicates an association between the regressor and missingness, meaning your data are MAR. 
4) If none are significant, then it's plausible that the data are missing completely at random. Because you are testing multiple hypotheses you will likely get some p-values that are small by chance. As a result you may need to adjust your cutoff for significance based on how many regressions you perform. 

Therefore, as in this case 12 out of 13 regressors have missing values, we should estimate 12 regressions where each feature with missing values is regressed on by the rest of the features, including the price.

Another way to test this is to check whether the columns with nulls (excluding the number of supermarkets) have an average price different than the observations which don't have nulls (Miguel Conner). If there is a significant difference, this could signal that rows with null values are following a different pattern than those that don't have null values.

In [10]:
# Identify columns that may contain nulls (excluding 'num_supermarkets')
columns_with_nulls = ['num_baths', 'square_meters', 'orientation', 'year_built', 
                      'is_furnished', 'has_pool', 'neighborhood', 
                      'num_crimes', 'has_ac', 'accepts_pets', 'floor']

# Create a boolean mask for rows with any nulls in the specified columns
mask_with_nulls = df3[columns_with_nulls].isnull().any(axis=1)

# Separate the DataFrame into two groups
group_with_nulls = df3[mask_with_nulls]
group_without_nulls = df3[~mask_with_nulls]

# Calculate average prices
avg_price_with_nulls = group_with_nulls['price'].mean()
avg_price_without_nulls = group_without_nulls['price'].mean()

# Display the results
print(f"Average price of observations with nulls: {avg_price_with_nulls}")
print(f"Average price of observations without nulls: {avg_price_without_nulls}")

Average price of observations with nulls: 1107.3341232227488
Average price of observations without nulls: 1093.981401384083


We can now check if the difference is statistically significant:

In [11]:
# Perform a t-test
t_stat, p_value = stats.ttest_ind(
    group_with_nulls['price'],
    group_without_nulls['price'],
    equal_var=False  # Use False if you assume unequal variance
)

print(f"T-statistic: {t_stat}, P-value: {p_value}")

# Determine significance level (e.g., alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("The difference in average prices is statistically significant at alpha", alpha)
else:
    print("The difference in average prices is not statistically significant at alpha", alpha)

T-statistic: 2.164010231366908, P-value: 0.030496436700082827
The difference in average prices is statistically significant at alpha 0.05


***Since the difference in average prices of observations with null values and those that don't have null values is statistically significant, data cannot be assumed to be MCAR (so listwise deletion is not the way to go)!***

What happens if we exclude the columns `num_supermarkets` AND `orientation`?

In [12]:
# Identify columns that may contain nulls (excluding 'num_supermarkets')
columns_with_nulls = ['num_baths', 'square_meters', 'year_built', 
                      'is_furnished', 'has_pool', 'neighborhood', 
                      'num_crimes', 'has_ac', 'accepts_pets', 'floor']

# Create a boolean mask for rows with any nulls in the specified columns
mask_with_nulls = df3[columns_with_nulls].isnull().any(axis=1)

# Separate the DataFrame into two groups
group_with_nulls = df3[mask_with_nulls]
group_without_nulls = df3[~mask_with_nulls]

# Calculate average prices
avg_price_with_nulls = group_with_nulls['price'].mean()
avg_price_without_nulls = group_without_nulls['price'].mean()

# Display the results
print(f"Average price of observations with nulls: {avg_price_with_nulls}")
print(f"Average price of observations without nulls: {avg_price_without_nulls}")

Average price of observations with nulls: 1114.2196610169492
Average price of observations without nulls: 1096.3150957854407


In [13]:
# Perform a t-test
t_stat, p_value = stats.ttest_ind(
    group_with_nulls['price'],
    group_without_nulls['price'],
    equal_var=False  # Use False if you assume unequal variance
)

print(f"T-statistic: {t_stat}, P-value: {p_value}")

# Determine significance level (e.g., alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("The difference in average prices is statistically significant at alpha", alpha)
else:
    print("The difference in average prices is not statistically significant at alpha", alpha)

T-statistic: 2.290535539337071, P-value: 0.02208471166633584
The difference in average prices is statistically significant at alpha 0.05


***We get the same result, so removing orientation does NOT solve the problem!!!***

#### ii. Listwise deletion
Only applicable if data is MCAR (if not, deletion could be biased). If all rows with NaNs except for the ones in num_supermarkets are dropped, is the number of observations reduced too much?

In [14]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8000 non-null   int64  
 1   num_rooms         8000 non-null   float64
 2   num_baths         7840 non-null   float64
 3   square_meters     7845 non-null   float64
 4   orientation       5659 non-null   object 
 5   year_built        7830 non-null   float64
 6   is_furnished      7835 non-null   object 
 7   has_pool          7844 non-null   object 
 8   neighborhood      7835 non-null   object 
 9   num_crimes        7840 non-null   float64
 10  has_ac            7831 non-null   object 
 11  accepts_pets      7845 non-null   object 
 12  num_supermarkets  1411 non-null   float64
 13  price             8000 non-null   int64  
 14  floor             7851 non-null   object 
dtypes: float64(6), int64(2), object(7)
memory usage: 937.6+ KB


In [15]:
def listwise_deletion(df: pd.DataFrame, column_to_exclude: str):
        df_listwise_deleted = df.dropna(subset=[col for col in df.columns if col != column_to_exclude])
        print(f'Number of dropped observations: {df.shape[0] - df_listwise_deleted.shape[0]}, or {(df.shape[0] - df_listwise_deleted.shape[0]) * 100 / df.shape[0]}%')
        return df_listwise_deleted

First, we apply listwise deletion to the data frame before applying OHE, so that we can more easily delete columns than after applying OHE:

In [16]:
df3_listwise_deleted = listwise_deletion(df = df3, column_to_exclude = 'num_supermarkets')
df3_listwise_deleted

Number of dropped observations: 3376, or 42.2%


,id,num_rooms,num_baths,square_meters,orientation,year_built,is_furnished,has_pool,neighborhood,num_crimes,has_ac,accepts_pets,num_supermarkets,price,floor
1,1562,4.0,1.0,133.0,west,1960.0,False,False,Sants,4.0,False,False,2.0,1396,7
5,5934,4.0,2.0,77.0,west,1987.0,True,True,Eixample,0.0,False,False,3.0,760,1
8,3509,1.0,1.0,59.0,south,1969.0,True,False,Gràcia,0.0,False,False,NaN,933,5
10,5867,3.0,3.0,101.0,south,2014.0,False,False,Gràcia,9.0,False,True,NaN,1124,7
11,169,3.0,2.0,115.0,south,2018.0,False,False,Sarrià-Sant Gervasi,0.0,False,True,NaN,1005,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7992,4427,3.0,2.0,125.0,north,1996.0,False,False,Sant Martí,4.0,True,False,NaN,1263,10
7993,467,4.0,1.0,157.0,south,1992.0,True,False,Eixample,0.0,False,True,NaN,1709,6
7994,6266,3.0,3.0,62.0,south,1979.0,True,True,Sarrià-Sant Gervasi,0.0,True,True,NaN,928,2
7997,5391,4.0,1.0,89.0,east,2015.0,True,False,Sant Martí,0.0,True,False,1.0,1220,9


42.2% of the observations are dropped if all null values of the features (except for the number of supermarkets) are dropped. If we assume that the number of supermarkets close is not an important predicting feature (which will be checked later), we can drop this column as there are too much null values.

In [17]:
df3_listwise_deleted_nosupm = df3_listwise_deleted.dropna(axis = 1)
df3_listwise_deleted_nosupm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4624 entries, 1 to 7998
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4624 non-null   int64  
 1   num_rooms      4624 non-null   float64
 2   num_baths      4624 non-null   float64
 3   square_meters  4624 non-null   float64
 4   orientation    4624 non-null   object 
 5   year_built     4624 non-null   float64
 6   is_furnished   4624 non-null   object 
 7   has_pool       4624 non-null   object 
 8   neighborhood   4624 non-null   object 
 9   num_crimes     4624 non-null   float64
 10  has_ac         4624 non-null   object 
 11  accepts_pets   4624 non-null   object 
 12  price          4624 non-null   int64  
 13  floor          4624 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 541.9+ KB


Now, we apply OHE to the resulting data frame:

In [18]:
df4 = ohe_join(dataframe = df3_listwise_deleted_nosupm,
               variables_to_encode = ['orientation', 'neighborhood', 'floor'],
               na_column = False)

We set the dummy columns to booleans:

In [19]:
for column in ['is_furnished', 'has_pool', 'has_ac', 'accepts_pets']:
        df4[column] = df4[column].astype(dtype = bool)

In [20]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4624 entries, 1 to 7998
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                4624 non-null   int64  
 1   num_rooms                         4624 non-null   float64
 2   num_baths                         4624 non-null   float64
 3   square_meters                     4624 non-null   float64
 4   year_built                        4624 non-null   float64
 5   is_furnished                      4624 non-null   bool   
 6   has_pool                          4624 non-null   bool   
 7   num_crimes                        4624 non-null   float64
 8   has_ac                            4624 non-null   bool   
 9   accepts_pets                      4624 non-null   bool   
 10  price                             4624 non-null   int64  
 11  orientation_east                  4624 non-null   bool   
 12  orientation

Now, we could create a new list with the categorical features of df4, to access them more easily:

In [21]:
# Lists to store column names
floor_dummies = [f'floor_{str(x)}' for x in range(1, 11)]
neighborhood_dummies = ['neighborhood_Ciutat Vella', 'neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 
                        'neighborhood_Les Cors', 'neighborhood_Nou Barris',
                        'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 
                        'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi']
orientation_dummies = ['orientation_east', 'orientation_north', 'orientation_south', 'orientation_west']

categorical_features_df4 = ['is_furnished', 'has_pool', 'has_ac','accepts_pets', 
                            orientation_dummies, neighborhood_dummies,floor_dummies]

categorical_features_df4

['is_furnished',
 'has_pool',
 'has_ac',
 'accepts_pets',
 ['orientation_east',
  'orientation_north',
  'orientation_south',
  'orientation_west'],
 ['neighborhood_Ciutat Vella',
  'neighborhood_Eixample',
  'neighborhood_Gràcia',
  'neighborhood_Horta',
  'neighborhood_Les Cors',
  'neighborhood_Nou Barris',
  'neighborhood_Sant Andreu',
  'neighborhood_Sant Martí',
  'neighborhood_Sants',
  'neighborhood_Sarrià-Sant Gervasi'],
 ['floor_1',
  'floor_2',
  'floor_3',
  'floor_4',
  'floor_5',
  'floor_6',
  'floor_7',
  'floor_8',
  'floor_9',
  'floor_10']]

In [22]:
print(len(categorical_features_df4))

7


And we also subtract the `num_supermarkets` variable from the numerical_features list:

In [23]:
numerical_features_df4 = ['num_rooms',
 'num_baths',
 'square_meters',
 'year_built',
 'num_crimes']
numerical_features_df4

['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes']

Finally, we create a list which contains all of the features:

In [24]:
features_df4 = numerical_features_df4 + categorical_features_df4
print(features_df4)
print(len(features_df4))

['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes', 'is_furnished', 'has_pool', 'has_ac', 'accepts_pets', ['orientation_east', 'orientation_north', 'orientation_south', 'orientation_west'], ['neighborhood_Ciutat Vella', 'neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 'neighborhood_Les Cors', 'neighborhood_Nou Barris', 'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi'], ['floor_1', 'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7', 'floor_8', 'floor_9', 'floor_10']]
12


##### Preprocessing dataset 1: OHE applied, listwise deletion with no num_supermarkets and keeping outliers of num_rooms 

In [30]:
# df4.to_csv('2_preprocessed_datasets/v1_prepr_ohe-listwise_deletion-no_num_supm-with_num_rooms_outliers.csv', index = False)

##### Preprocessing dataset 2: OHE applied, no listwise deletion, no num_supermarkets and keeping outliers of num_rooms

In [26]:
df5 = df3.copy()

df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8000 non-null   int64  
 1   num_rooms         8000 non-null   float64
 2   num_baths         7840 non-null   float64
 3   square_meters     7845 non-null   float64
 4   orientation       5659 non-null   object 
 5   year_built        7830 non-null   float64
 6   is_furnished      7835 non-null   object 
 7   has_pool          7844 non-null   object 
 8   neighborhood      7835 non-null   object 
 9   num_crimes        7840 non-null   float64
 10  has_ac            7831 non-null   object 
 11  accepts_pets      7845 non-null   object 
 12  num_supermarkets  1411 non-null   float64
 13  price             8000 non-null   int64  
 14  floor             7851 non-null   object 
dtypes: float64(6), int64(2), object(7)
memory usage: 937.6+ KB


In [27]:
# Drop number of supermarkets column
df5.drop(labels = 'num_supermarkets', axis = 1, inplace = True)

# Apply OHE
df5 = ohe_join(dataframe = df5,
               variables_to_encode = ['orientation', 'neighborhood', 'floor'],
               na_column = True)

df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8000 non-null   int64  
 1   num_rooms                         8000 non-null   float64
 2   num_baths                         7840 non-null   float64
 3   square_meters                     7845 non-null   float64
 4   year_built                        7830 non-null   float64
 5   is_furnished                      7835 non-null   object 
 6   has_pool                          7844 non-null   object 
 7   num_crimes                        7840 non-null   float64
 8   has_ac                            7831 non-null   object 
 9   accepts_pets                      7845 non-null   object 
 10  price                             8000 non-null   int64  
 11  orientation_east                  8000 non-null   bool   
 12  orient

In [31]:
# df5.to_csv('2_preprocessed_datasets/v2_prepr_ohe_with_na_dummies-no_num_supm-with_num_rooms_outliers.csv', index = False)

#### iii. Imputing the median
If done, done inside the pipeline of training (after splitting). See training notebook.

#### iv. KNN imputer
If done, done inside the pipeline of training (after splitting). See training notebook.

### 2.4.2. Handling outliers
Remember to handle outliers of `num_rooms`!!

In [ ]:
def detect_outliers_iqr(data, column_name, threshold=1.5):
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    outliers = data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)]

    return outliers

def impute_outliers_with_median(data, column_name, outliers):
    # Calculate the median excluding outliers
    filtered_data = data[~data.index.isin(outliers.index)]
    median_without_outliers = filtered_data[column_name].median()

    # Impute the mean to the outliers
    data.loc[outliers.index, column_name] = median_without_outliers

    return data, median_without_outliers

In [ ]:
# Detect outliers in column 'num_rooms'
outliers = detect_outliers_iqr(df7, 'num_rooms')

# Impute outliers with the mean of column 'A' (excluding outliers)
data, median_num_rooms_no_outliers = impute_outliers_with_median(df7, 'num_rooms', outliers)

data

## 2.5. Feature scaling and normalization
To be able to compare the magnitudes of the coefficients of the linear regression model and to select features based on this criteria, it is necessary to standardize the data. According to Anna, it is NOT necessary to standardize dummy variables. Apply the same transformation of the training data to the test data, but separately!
 - Standardization is also essential before performing regularization methods like LASSO and Ridge.
 - `StandardScaler()` from scikit-learn.

### 2.5.1. Natural log transformation
This transformation works well for:
- right-skewed data,
- compressed data, and 
- data with large outliers. 

After we log transform our data, one large benefit is that it will allow the data to be closer to a “normal”  distribution. It also changes the scale so our data points will drastically reduce the range of their values. But leep in mind, just because your data is skewed does not mean that a log transformation is the best answer. You would not want to log transform your feature if:

1. You have values less than 0. The natural logarithm of a negative number is undefined.
2. You have left-skewed data. That data may call for a square or cube transformation.
3. You have non-parametric data.

No log transformation applied here.

### 2.5.2. Standardization
If done, done inside the pipeline of training (after splitting). See training notebook.